## Dataset Loading
This section imports the required libraries and loads the dataset into a pandas DataFrame. The dataset is the foundation for all subsequent preprocessing and analysis steps. we will install all the librabies that we need.

In [2]:
%pip install statsmodels pandas matplotlib seaborn

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-2.3.3-cp39-cp39-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached matplotlib-3.9.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached scipy-1.13.1-cp39-cp39-macosx_12_0_arm64.whl.metadata (60 kB)
  Using cached patsy-1.0.2-py2.py3-none-any.whl.metadata (3.6 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached contourpy-1.3.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.60.2-cp39-cp39-macosx_10_9_universal2.whl.metadata (113 kB)
  Using cached kiwisolver-1.4.7-cp39-cp39-macosx_11_0_arm64.whl.metadata (6.3 kB)
  Using cached pillow-11.3.0-cp39

## 1️⃣ Loading & Saving Data

### `pd.read_csv()` — Load data
Reads a CSV file into a DataFrame.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


df = pd.read_csv("./US_Regional_Sales_Data.csv", encoding='latin1')


df.info()



# 2️⃣ Data filteration 


### 1.1 Droping all the columns that are not required
 - here we will drop all the location related location , except the country and city 
 - i have to also removed all proof related info

In [ ]:
df = df.drop(columns=[ "ï»¿OrderNumber","WarehouseCode","ShipDate","DeliveryDate","_SalesTeamID","_CustomerID","_StoreID","_ProductID",])
df_date = pd.to_datetime(df['OrderDate'])
df.isnull().sum()


### Changing the data type of discounted price and actual price

### 1.3 Indexing the data 

- we will index the "Created" columns into a index
- The created is of String type , so we need to convert it to datetime type
- The index we have created is type of index not a datetimeindex


In [ ]:
df1 = df.copy()
df1.set_index("OrderDate", inplace=True)
df1.index = pd.to_datetime(df1.index)
df1.resample("ME").mean(numeric_only=True)



## 1.4 FINAL - Data information 
- we have two differnet data now , "DF" where all discounted products information is collected
- **df1** - where we have index created for the datetime index.
- 7 days rollings statics and 30 days rolling statatics will be used , where index will helped us  or basically **df1**

## 3️⃣Exploratory Price and Discount Analysis

This section performs exploratory data analysis to examine the relationship between product prices, discounts, and sales channels over time.

First, the order date is converted to a datetime format, allowing temporal analysis. Price-related columns are cleaned by removing thousand separators and converting the values to numeric format to ensure accurate mathematical operations.

The analysis then visualizes key relationships using scatter plots and box plots:

- The relationship between order date and unit price is examined to identify temporal trends and price dispersion over time.
- Discounts applied over time are visualized to observe potential promotional patterns and seasonal effects.
- The relationship between unit price and unit cost i


In [ ]:



# extract year
df["year"] = pd.to_datetime(df["OrderDate"]).dt.year
df["Unit Price"] = (
    df["Unit Price"]
    .str.replace(",", "", regex=False)
    .astype(float)
)
df["Unit Cost"] = (
    df["Unit Cost"]
    .str.replace(",", "", regex=False)
    .astype(float)
)
df["OrderDate"] = pd.to_datetime(df["OrderDate"], format="%d/%m/%y")


fig, axes = plt.subplots(2, 2, figsize=(26, 30))
fig.suptitle("Price & Discount Analysis", fontsize=32)

# ---------------- Plot 1: Year vs Unit Price ----------------
axes[0, 0].scatter(df["OrderDate"], df["Unit Price"], alpha=0.3)
axes[0, 0].set_title("Year vs Unit Price", fontsize=18)
axes[0, 0].set_ylabel("Unit Price")

# ---------------- Plot 2: Date vs Discount Applied ----------------
axes[0, 1].scatter(df["OrderDate"], df["Discount Applied"], alpha=0.3)
axes[0, 1].set_title("Order Date vs Discount Applied", fontsize=18)
axes[0, 1].tick_params(axis="x", rotation=45)

# ---------------- Plot 3: Unit Price vs Unit Cost ----------------
axes[1, 0].scatter(df["Unit Price"], df["Unit Cost"], alpha=0.3)
axes[1, 0].set_title("Unit Price vs Unit Cost", fontsize=18)
axes[1, 0].set_xlabel("Unit Price")
axes[1, 0].set_ylabel("Unit Cost")

# ---------------- Plot 4: Discount vs Sales Channel (BOXPLOT) ----------------
df.boxplot(
    column="Discount Applied",
    by="Sales Channel",
    ax=axes[1, 1]
)
axes[1, 1].set_title("Discount Applied by Sales Channel", fontsize=18)
axes[1, 1].set_xlabel("Sales Channel")
axes[1, 1].set_ylabel("Discount Applied")

plt.suptitle("")  # remove pandas auto-title
plt.tight_layout()
plt.show()


### 2.3 Checking seasonlaity and pattern 
 - while averaging the discounted prices base on the montly there was some difference betwe_en the discounted_percent average 
 - **1st problem** - There was difference between the averaging for the **df_low** and **df_high** .


In [ ]:

df["OrderDate"] = pd.to_datetime(df["OrderDate"], format="%d/%m/%y")



yearly_low = df.groupby(df["OrderDate"].dt.month)["Discount Applied"].mean()
yearly_low.plot(kind="bar", title="Average Discount % per Month (Price ≤ 1000)", figsize=(10, 6))
plt.xlabel("Month")
plt.ylabel("Average Discount %")
plt.show()




### Visualization after removing some noises 
As we have remove the dat from year 2016 to 2022 , now we have the data from 2023 to 2025 , for which we will check the seasolity pattern. Furthermore , for the Sarima model,, we just need that.

In [ ]:
df["OrderDate"] = pd.to_datetime(df["OrderDate"], format="%Y-%m-%d")

yearly_low = df.groupby(df["OrderDate"].dt.month)["Discount Applied"].mean()
yearly_low.plot(kind="bar", title="Average Discount % per Month (Price ≤ 1000)", figsize=(10, 6))
plt.xlabel("Month")
plt.ylabel("Average Discount %")
plt.show()

In [ ]:
df_2018 = df[df['OrderDate'].dt.year == 2018]
df_2019 = df[df['OrderDate'].dt.year == 2019]
df_2020 = df[df['OrderDate'].dt.year == 2020]

yearly_2018 = df_2018.groupby(df_2018["OrderDate"].dt.month)["Discount Applied"].mean()
yearly_2019 = df_2019.groupby(df_2019["OrderDate"].dt.month)["Discount Applied"].mean()
yearly_2020 = df_2020.groupby(df_2020["OrderDate"].dt.month)["Discount Applied"].mean()


df_years = pd.DataFrame({
    "2018": yearly_2018,
    "2019": yearly_2019,
    "2020": yearly_2020
})


df_years.plot(
    kind="bar",
    figsize=(12, 6),
    title="Average Discount(price<1000) % per Month (2023–2025)"
)

plt.xlabel("Month")
plt.ylabel("Average Discount %")
plt.legend(title="Year")
plt.xticks(rotation=0)
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))

for year in df_years.columns:
    plt.plot(
        df_years.index,
        df_years[year],
        marker="o",
        label=year
    )

plt.title("Average Discount % per Month (2023–2025)")
plt.xlabel("Month")
plt.ylabel("Average Discount %")
plt.xticks(range(1, 13))
plt.legend(title="Year")
plt.grid(alpha=0.3)
plt.show()


## Weeknd and weekdays 

1️⃣ Make sure your date column is datetime

In [35]:
df["OrderDate"] = pd.to_datetime(df["OrderDate"])


2️⃣ Create a weekday / weekend column

In [36]:
df["day_type"] = df["OrderDate"].dt.dayofweek.apply(
    lambda x: "Weekend" if x >= 5 else "Weekday"
)

3️⃣ Compare Weekday vs Weekend (mean example)

In [ ]:
df.groupby("day_type")["Unit Price"].mean()


4️⃣ Visualization options

In [ ]:
df.boxplot(
    column="Unit Price",
    by="day_type",
    figsize=(8, 5)
)
plt.title("Unit Price: Weekday vs Weekend")
plt.suptitle("")
plt.xlabel("Day Type")
plt.ylabel("Unit Price")
plt.show()


In [ ]:
df["OrderDate"] = pd.to_datetime(df["OrderDate"])
df_ts = df.set_index("OrderDate")

# daily average discount
daily_discount = df_ts["Discount Applied"].resample("D").mean()

plt.figure(figsize=(12, 4))
daily_discount.plot()
plt.title("Daily Average Discount Percentage")
plt.ylabel("Discount %")
plt.show()



### Rolling mean (smooths noise)
- If peaks repeat every ~7 days → weekly cycle

In [ ]:
daily_discount.rolling(window=7).mean.plot(figure=(12, 4))
plt.title("7-days Rolling Average of discounts over time")
plt.show()

### 4️⃣ Autocorrelation (MOST IMPORTANT)
- This is the main statistical tool for cycle detection.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

plot_acf(daily_discount.dropna(), lags=20)
plt.show()


### Seasonal decomposition (clear cycles)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition = seasonal_decompose(
    daily_discount.dropna(),
    model="additive",
    period=7   # try 7, 30, or 365
)

decomposition.plot()
plt.show()
